## My first Kaggle kernel. Your suggestions and comments will help me improve. ##

In [ ]:
import os
import warnings
warnings.simplefilter("ignore")

import cv2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df_train = pd.read_csv("../input/siim-isic-melanoma-classification/train.csv")
df_test  = pd.read_csv("../input/siim-isic-melanoma-classification/test.csv")

print("Train shape:", df_train.shape)
print("Test shape:", df_test.shape)

In [ ]:
df_train.sample(5)

In [ ]:
df_test.sample(5)

**Test set doesn't have diagnosis and benign_malignant columns.**

In [ ]:
unique_patient_train = df_train.patient_id.unique()
unique_patient_test = df_test.patient_id.unique()

print("Unique patients in training set:", unique_patient_train.shape[0])
print("Unique patients in test set:", unique_patient_test.shape[0])

In [ ]:
print("No of patients common in train and test sets:", np.intersect1d(unique_patient_train, unique_patient_test).shape[0])

Since the test set doesn't have diagnosis and benign_malignant, let's see what's there on those two columns.

In [ ]:
df_train.diagnosis.value_counts()

In [ ]:
# Set the value of "unknown" as nan
df_train.diagnosis = df_train.diagnosis.apply(lambda x: np.nan if x == "unknown" else x)

In [ ]:
df_train.diagnosis.value_counts()

In [ ]:
df_train.benign_malignant.value_counts()

In [ ]:
df_train.benign_malignant.isnull().sum()

In [ ]:
df_train.target.value_counts()

In [ ]:
ax = sns.countplot(x = "benign_malignant", hue = "target", data = df_train)

So the benign has been encoded as target = 0 and malignant has been encoded as target = 1. So we can safely remove the column benign_malignant.

In [ ]:
df_train.drop("benign_malignant", axis = 1, inplace = True)

In [ ]:
df_train.isnull().sum()

In [ ]:
# Let's drop the diagnosis as well as it has many null entry and is not present in the test set
df_train.drop("diagnosis", axis = 1, inplace = True)

In [ ]:
print("Train shape:", df_train.shape)
print("Test shape:", df_test.shape)

In [ ]:
df_train.isnull().sum().to_frame()

In [ ]:
df_test.isnull().sum().to_frame()

In [ ]:
(df_train.sex.value_counts() * 100 / df_train.shape[0]).to_frame()

In [ ]:
(df_test.sex.value_counts() * 100 / df_test.shape[0]).to_frame()

In [ ]:
# Since male and female ratio is almost same, with mens are more in number, let's fill those 65 missing items as male
df_train.sex = df_train.sex.fillna("male")
(df_train.sex.value_counts() * 100 / df_train.shape[0]).to_frame()

In [ ]:
ax = sns.countplot(x = "sex", hue = "target", data = df_train)

In [ ]:
# For age_approx, fill the na values with 0.0
df_train.age_approx = df_train.age_approx.fillna(0.0)

In [ ]:
fig = plt.figure(figsize = (10, 5))
ax = sns.countplot(x = "age_approx", hue = "target", data = df_train)

In [ ]:
(df_train.anatom_site_general_challenge.value_counts() * 100 / df_train.shape[0]).to_frame()

In [ ]:
(df_test.anatom_site_general_challenge.value_counts() * 100 / df_test.shape[0]).to_frame()

In [ ]:
# anatom_site_general_challenge needs to be imputed for both train and test. Let's use something called unknown for now
df_train.anatom_site_general_challenge = df_train.anatom_site_general_challenge.fillna("unknown")
df_test.anatom_site_general_challenge = df_test.anatom_site_general_challenge.fillna("unknown")

In [ ]:
df_train.info()

In [ ]:
# Let's do one hot encoding on sex column
df_train["sex_cat_m"] = 0
df_train.loc[df_train.sex == "male", "sex_cat_m"] = 1
df_train.drop("sex", axis = 1, inplace = True)

In [ ]:
# Let's do one hot encoding on anatom_site_general_challenge column
# torso, lower extremity, upper extremity, head/neck, unknown, palms/soles, oral/genital

temp = pd.get_dummies(df_train.anatom_site_general_challenge, prefix = "location")
temp.drop("location_unknown", axis = 1, inplace = True)
temp.columns = [("location_cat_" + str(i)) for i in range(1, 7)]

df_train = pd.concat([df_train, temp], axis = 1)
df_train.drop("anatom_site_general_challenge", axis = 1, inplace = True)

In [ ]:
temp = pd.get_dummies(df_test.anatom_site_general_challenge, prefix = "location")
temp.drop("location_unknown", axis = 1, inplace = True)
temp.columns = [("location_cat_" + str(i)) for i in range(1, 7)]

df_test = pd.concat([df_test, temp], axis = 1)
df_test.drop("anatom_site_general_challenge", axis = 1, inplace = True)

In [ ]:
values = df_train.target.values
df_train.drop("target", axis = 1, inplace = True)
df_train["target"] = values

In [ ]:
print("Train shape:", df_train.shape)
print("Test shape:", df_test.shape)

In [ ]:
df_train.corr().style.background_gradient(cmap = "RdBu")

**Somehow can we impute the missing entries?**

In [ ]:
df_train.target.value_counts().to_frame()

In [ ]:
(df_train.target.value_counts() * 100 / df_train.shape[0]).to_frame()

The dataset is very much imbalanced.

In [ ]:
df_train

**How does the images looks like?**

In [ ]:
def plot_images(df, n_rows = 5, n_cols = 5, figsize = (20, 20), resize = (1024, 1024), preprocessing = None, label = 0):
    query_string = "target == {}".format(label)
    df = df.query(query_string).reset_index(drop = True)
    fig = plt.figure(figsize = figsize)
    ax  = []
    base_path = "../input/siim-isic-melanoma-classification/jpeg/train/"

    for i in range(n_rows * n_cols):
        img = plt.imread(base_path + df.loc[i, "image_name"] + ".jpg")
        img = cv2.resize(img, resize)

        if preprocessing:
            img = preprocessing(img)

        ax.append(fig.add_subplot(n_rows, n_cols, i + 1) )
        plot_title = "Image {}: {}".format(str(i + 1), "Benign" if label == 0 else "Malignant") 
        ax[-1].set_title(plot_title)
        plt.imshow(img, alpha = 1, cmap = "gray")

    plt.show()

In [ ]:
# Training images - Benign
plot_images(df_train, label = 0, resize = (224, 224))

In [ ]:
# Training images - Malignant
plot_images(df_train, label = 1, resize = (224, 224))

#### Observations: ####
 - Dataset is very much imbalanced. Need external data for malignant cases.
 - anatom_site_general_challenge, age and sex need to be imputed and make those part of split/model.
 - In training set, there are 33126 images where there are 2056 unique patients. So on an average, there are around 15 images per patient. So while splitting the dataset for CV, need to take care of this fact.
 - Scaling the images as 224 x 224, instead of 1024 x 1024 can be done.